In [9]:
import pandas as pd
import numpy as np
import matplotlib as plt

In [10]:
df = pd.read_csv('data/final.csv')

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44738 entries, 0 to 44737
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   company_permalink  44738 non-null  object 
 1   company_name       44737 non-null  object 
 2   homepage_url       41971 non-null  object 
 3   category_list      42309 non-null  object 
 4   funding_total_usd  44738 non-null  object 
 5   status             44738 non-null  object 
 6   country_code       39748 non-null  object 
 7   state_code         38563 non-null  object 
 8   region             38926 non-null  object 
 9   city               38928 non-null  object 
 10  funding_rounds     44738 non-null  int64  
 11  founded_at         33318 non-null  object 
 12  first_funding_at   44718 non-null  object 
 13  last_funding_at    44738 non-null  object 
 14  is_roundA          44738 non-null  float64
 15  is_roundB          44738 non-null  float64
 16  is_roundC          447

In [12]:
df.isnull().sum()

company_permalink        0
company_name             1
homepage_url          2767
category_list         2429
funding_total_usd        0
status                   0
country_code          4990
state_code            6175
region                5812
city                  5810
funding_rounds           0
founded_at           11420
first_funding_at        20
last_funding_at          0
is_roundA                0
is_roundB                0
is_roundC                0
is_roundD                0
is_roundE                0
is_roundF                0
is_roundG                0
is_roundH                0
investor_count           0
dtype: int64

In [13]:
df.drop(['founded_at','company_permalink','homepage_url','region','city','state_code'],inplace=True,axis=1)

In [14]:
df.isnull().sum()

company_name            1
category_list        2429
funding_total_usd       0
status                  0
country_code         4990
funding_rounds          0
first_funding_at       20
last_funding_at         0
is_roundA               0
is_roundB               0
is_roundC               0
is_roundD               0
is_roundE               0
is_roundF               0
is_roundG               0
is_roundH               0
investor_count          0
dtype: int64

In [15]:
df.dropna(inplace=True)

In [16]:
df.isnull().sum()

company_name         0
category_list        0
funding_total_usd    0
status               0
country_code         0
funding_rounds       0
first_funding_at     0
last_funding_at      0
is_roundA            0
is_roundB            0
is_roundC            0
is_roundD            0
is_roundE            0
is_roundF            0
is_roundG            0
is_roundH            0
investor_count       0
dtype: int64

In [17]:
df.shape[0]

38571

EDA